## Multilabel 

In [ ]:
import tensorflow as tf
import cv2
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
import keras as K
import matplotlib.cm as cm
from tqdm import tqdm
from pathlib import Path
from keras import backend as K
from sklearn.model_selection import train_test_split
from PIL import Image
from sklearn.metrics import multilabel_confusion_matrix, ConfusionMatrixDisplay
# from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.losses import binary_crossentropy
from tensorflow import keras
from sklearn.model_selection import KFold, StratifiedKFold
from keras.callbacks import TensorBoard
physical_devices = tf.config.list_physical_devices('GPU')
try:
  tf.config.experimental.set_memory_growth(physical_devices[0], True)
except:
  # Invalid device or cannot modify virtual devices once initialized.
  pass

# Prepare Dataset

In [ ]:
df = pd.read_csv('data/label.csv')
df

In [ ]:
# buat daftar unique dari semua label
unique_label = set()
for label in df["Label"]:
    label_list = label.strip("[]").split(",")
    for label in label_list:
        unique_label.add(label.strip())

unique_label = sorted(list(unique_label))
# buatkan kolom one-hot encodinf setiap label
for label in unique_label:
    df[label] = df["Label"].apply(lambda x: int(label in x))

In [ ]:
df.head()

In [ ]:
len(df)

In [ ]:
df["Filename"] = "data/image/" + df["Filename"] 

In [ ]:
df['Label'].value_counts().plot(kind='bar')

In [ ]:
random_index = np.random.randint(0, len(df), 10)
fig, axes = plt.subplots(nrows=2, ncols=4, figsize=(10, 10),
                    subplot_kw={'xticks': [], 'yticks': []})
for i, ax in enumerate(axes.flat):
    ax.imshow(plt.imread(df.Filename[random_index[i]]))
    ax.set_title(df.Label[random_index[i]])
fig.suptitle("Tedong Multilabel", fontsize=16)
plt.show()
fig.savefig("tedong.png")

In [ ]:
# df.drop(columns='Label', axis=1, inplace=True)
# df

In [ ]:
train, test = train_test_split(df, test_size=0.05, random_state=42)

In [ ]:
train = train.drop(columns="Label", axis=1)
train

In [ ]:
test = test.drop(columns="Label", axis=1)
test

In [ ]:
classes = ["Hewan liar","Kerbau","Manusia","Motor","Truk"]

In [ ]:
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255, validation_split=0.2)
test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
# The value for class_mode in flow_from_dataframe MUST be 'raw' if you are attempting to do multilabel classification.
train_gen = train_datagen.flow_from_dataframe(train, 
                                            x_col='Filename', 
                                            y_col=classes,
                                            target_size=(224,224),
                                            class_mode='raw',
                                            batch_size=32,
                                            shuffle=True,
                                            subset='training')
val_gen = train_datagen.flow_from_dataframe(train,
                                            x_col='Filename',
                                            y_col=classes,
                                            target_size=(224,224),
                                            class_mode='raw',
                                            batch_size=16,
                                            subset='validation')

In [ ]:
test_gen = test_datagen.flow_from_dataframe(test,
                                            x_col='Filename',
                                            # y_col="Label",
                                            batch_size=1,
                                            shuffle=None,
                                            seed=42,
                                            target_size=(224,224),
                                            class_mode=None)

## Metrics

In [ ]:
def f1_score(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives =K.sum(K.round(K.clip(y_pred, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    precision = true_positives / (predicted_positives + K.epsilon())
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

metrics = ["accuracy", 
           tf.keras.metrics.Recall(),
           tf.keras.metrics.Precision(),
           f1_score]

## Callback

In [ ]:
import datetime
def tensorboard_callback(name):
    logdir = os.path.join("Tensorboard/logs", datetime.datetime.now().strftime(f"%Y-%m-%d-{name}"))
    return TensorBoard(logdir)

class myCallback(tf.keras.callbacks.Callback):
    def __init__(self, accuracy=0.9):
        self.accuracy = accuracy
        
    def on_epoch_end(self, epoch, logs=None):
        if logs.get("val_accuracy") > self.accuracy:
            print(f"\nAkurasi telah mencapai {self.accuracy}%")
            self.model.stop_training = True

## Plot Model

In [ ]:
def plot_model(model, name_model):
        fig, (ax1, ax2, ax3) = plt.subplots(nrows=3,
                                     ncols=1,
                                     figsize=(15, 15))
        ax1.plot(model.history["accuracy"], marker=".")
        ax1.plot(model.history["recall"], marker=".") 
        ax1.plot(model.history["precision"], marker=".")
        ax1.plot(model.history["f1_score"], marker=".")
        ax1.set_xlabel("epochs")
        ax1.legend(["accuracy", 
        "recall", 
        "precission", 
        "f1"], loc="lower left")
        ax1.grid(True)
        ax1.set_title(name_model + ' Training')
        
        ax2.plot(model.history["val_accuracy"], marker=".")
        ax2.plot(model.history["val_recall"], marker=".") 
        ax2.plot(model.history["val_precision"], marker=".")
        ax2.plot(model.history["val_f1_score"], marker=".")
        ax2.set_xlabel("epochs")
        ax2.legend(["val_accuracy", 
                "val_recall", 
                "val_precission", 
                "val_f1"], loc="lower left")
        ax2.grid(True)
        ax2.set_title(name_model + " Val")
        
        ax3.plot(model.history["loss"])
        ax3.plot(model.history["val_loss"])
        ax3.set_xlabel("epochs")
        ax3.legend(["loss", 'val_loss'])
        ax3.grid(True)
        ax3.set_title(name_model + " Loss")
        fig.suptitle(name_model, fontsize=24)
        return plt.show

## Predict

In [ ]:
def predict(model, name_file):
    test_gen.reset()
    pred = model.predict(test_gen,steps=test_gen.n // test_gen.batch_size, verbose=1)
    pred_bool = (pred > 0.5)
    predictions = pred_bool.astype(int)
    columns=classes
    #columns should be the same order of y_col
    results = pd.DataFrame(predictions, columns=columns)
    results["Filenames"] = test_gen.filenames
    ordered_cols=["Filenames"]+columns
    results=results[ordered_cols]
    return results.to_csv(name_file, index=False)
    

## visual confusion matrix

In [ ]:
y_true = test[classes]

In [ ]:
def visaul_confusion_matrix(y_true, y_pred):
    mcm = multilabel_confusion_matrix(y_true, y_pred)
    print(mcm)
# Display confusion matrices for each class
    for i in range(len(classes)):
        disp = ConfusionMatrixDisplay(confusion_matrix=mcm[i], display_labels=[0, 1])
        disp.plot(cmap='viridis', values_format='d')
        plt.title(f'Confusion Matrix for {classes[i]}')
    return plt.show()
        

# Model

### Resnet50V2

In [ ]:
def resnet():
    pre_trained_model = tf.keras.applications.resnet_v2.ResNet50V2(input_shape=(224, 224, 3),
                                                        include_top=False,
                                                        weights='imagenet')
    
    for layer in pre_trained_model.layers:
        layer.trainable = False
        
    return pre_trained_model

In [ ]:
resnet = resnet()

In [ ]:
resnet.summary()

In [ ]:
last_layer = resnet.get_layer("post_relu")
last_output = last_layer.output

In [ ]:
last_output

In [ ]:
def model_resnet(pre_trained_model, last_layer):
    x = tf.keras.layers.Flatten()(last_layer)
    x = tf.keras.layers.Dense(512, activation="relu")(x)
    x = tf.keras.layers.Dense(256, activation="relu")(x)
    x = tf.keras.layers.Dropout(0.3)(x)
    x = tf.keras.layers.Dense(5, activation="sigmoid")(x)
    
    model = tf.keras.Model(inputs=pre_trained_model.input, outputs=x)
    return model

In [ ]:
model_resNet50V2 = model_resnet(resnet, last_output)
model_resNet50V2.compile(optimizer='adam', loss="binary_crossentropy",
                   metrics=metrics)
model_resNet50V2.summary()

In [ ]:
with tf.device("/device:GPU:0"):
        histori1 = model_resNet50V2.fit(train_gen,
                              steps_per_epoch=train_gen.n // train_gen.batch_size,
                              epochs=30,
                              validation_data=val_gen,
                              validation_steps=val_gen.n // val_gen.batch_size,
                              callbacks=[tensorboard_callback("model_resNet50V2")])

In [ ]:
plot_model(histori1, "Model ResNet50V2")

In [ ]:
predict(model_resNet50V2, "model resNet50V2.csv")

In [ ]:
dt_predict_resnet = pd.read_csv("model resNet50V2.csv")

In [ ]:
y_pred_1 = dt_predict_resnet[classes]

In [ ]:
visaul_confusion_matrix(y_true, y_pred_1)

In [ ]:
for i

### Model Resnet101v2

In [ ]:
def resnet101v2():
    pre_train = tf.keras.applications.resnet_v2.ResNet101V2(
        input_shape=(224, 224, 3),
        include_top=False,
        weights="imagenet"
    )
    
    for layer in pre_train.layers:
        layer.trainable = False
        
    return pre_train

In [ ]:
resnet101v2 = resnet101v2()
resnet101v2.summary()

### Inception ResnetV2

In [ ]:
def inception_resnetv2():
    pre_trainded_model = tf.keras.applications.inception_resnet_v2.InceptionResNetV2(
        input_shape = (224, 224, 3),
        include_top = False,
        weights='imagenet'
    )
    
    for layer in pre_trainded_model.layers:
        layer.trainable = False
        
    return pre_trainded_model

In [ ]:
inception_resnetv2 = inception_resnetv2()
inception_resnetv2.summary()

In [ ]:
def model_inception_resnetv2(pre_trained_model, last_layer):
    x = tf.keras.layers.Flatten()(last_layer)
    x = tf.keras.layers.Dense(512, activation="relu")(x)
    x = tf.keras.layers.Dense(256, activation="relu")(x)
    x = tf.keras.layers.Dropout(0.3)(x)
    x = tf.keras.layers.Dense(5, activation="sigmoid")(x)
    
    model = tf.keras.Model(inputs=pre_trained_model.input, outputs=x)
    return model

In [ ]:
last_layer_inresnetv2 = inception_resnetv2.get_layer("conv_7b_ac")
last_output_inresnetv2 = last_layer_inresnetv2.output


In [ ]:
model_inception_resnetv2 = model_inception_resnetv2(inception_resnetv2, last_output_inresnetv2)
model_inception_resnetv2.compile(optimizer='adam', loss="binary_crossentropy",
                   metrics=metrics)

In [ ]:
model_inception_resnetv2.summary()

In [ ]:
with tf.device("/device:GPU:0"):
    history6 = model_inception_resnetv2.fit(train_gen,
                              steps_per_epoch=train_gen.n // train_gen.batch_size,
                              epochs=50,
                              validation_data=val_gen,
                              validation_steps=val_gen.n // val_gen.batch_size,
                              callbacks=[tensorboard_callback("model_inception_resnetv2"),
                              myCallback(accuracy=0.95)])

In [ ]:
plot_model(history6, "Model Inception-ResNetV2")

In [ ]:
predict(model_inception_resnetv2, "model_inception_resnetv2.csv")
dt_model_inresnetv2 = pd.read_csv("model_inception_resnetv2.csv")
y_pred_6 = dt_model_inresnetv2[["Hewan liar", "Motor", "Kerbau", "Manusia", "Truk"]]

In [ ]:
visaul_confusion_matrix(y_true, y_pred_6)

### EfficienNet

In [ ]:
def efficienNet():
    pre_trained_model = tf.keras.applications.efficientnet_v2.EfficientNetV2S(
        input_shape=(224, 224, 3),
        include_top=False,
        weights='imagenet'
    )
    
    for layer in pre_trained_model.layers:
        layer.trainable = False
    
    return pre_trained_model

In [ ]:
efficienNet = efficienNet()
efficienNet.summary()

In [ ]:
last_layer_efficientNet = efficienNet.get_layer("top_activation")
last_output_efficientNet = last_layer_efficientNet.output

In [ ]:
def final_efficientNet(pre_trained_model, last_layer):
    x = tf.keras.layers.Flatten()(last_layer)
    x = tf.keras.layers.Dense(512, activation="relu")(x)
    x = tf.keras.layers.Dense(256, activation="relu")(x)
    x = tf.keras.layers.Dropout(0.3)(x)
    x = tf.keras.layers.Dense(5, activation="sigmoid")(x)
    
    model = tf.keras.Model(inputs=pre_trained_model.input, outputs=x)  
    return model

In [ ]:
model_efficienNet = final_efficientNet(efficienNet, last_output_efficientNet)
model_efficienNet.compile(optimizer='adam', loss="binary_crossentropy",
                   metrics=metrics)

In [ ]:
model_efficienNet.summary()

In [ ]:
with tf.device("/device:GPU:0"):
    history5 = model_efficienNet.fit(train_gen,
                              steps_per_epoch=train_gen.n // train_gen.batch_size,
                              epochs=50,
                              validation_data=val_gen,
                              validation_steps=val_gen.n // val_gen.batch_size,
                              callbacks=[tensorboard_callback("model_efficienNetV2S"),
                              myCallback(accuracy=0.95)])

In [ ]:
plot_model(history5, "Model EfficienNetV2 Small")

In [ ]:
predict(model_efficienNet, 'model efficienNet.csv')

In [ ]:
dt_model_efficenNet = pd.read_csv('model efficienNet.csv')

In [ ]:
y_pred_5 = dt_model_efficenNet[["Hewan liar", "Motor", "Kerbau", "Manusia", "Truk"]]

In [ ]:
visaul_confusion_matrix(y_true, y_pred_5)

# Define Vis Model

In [ ]:
def layer_name(model):
    outputs = [layer.outputs for layer in model.layer]
    layer_name = []
    for layer in outputs:
        layer_name.append(layer.name.split("/"))
    
    return layer_name

In [ ]:
def visualize_intermediate_activations(layer_names, activations):
    assert len(layer_names)==len(activations), "Make sure layers and activation values match"
    images_per_row=16
    
    for layer_name, layer_activation in zip(layer_names, activations):
        nb_features = layer_activation.shape[-1]
        size= layer_activation.shape[1]

        nb_cols = nb_features // images_per_row
        grid = np.zeros((size*nb_cols, size*images_per_row))

        for col in range(nb_cols):
            for row in range(images_per_row):
                feature_map = layer_activation[0,:,:,col*images_per_row + row]
                feature_map -= feature_map.mean()
                feature_map /= feature_map.std()
                feature_map *=255
                feature_map = np.clip(feature_map, 0, 255).astype(np.uint8)

                grid[col*size:(col+1)*size, row*size:(row+1)*size] = feature_map

        scale = 1./size
        plt.figure(figsize=(scale*grid.shape[1], scale*grid.shape[0]))
        plt.title(layer_name)
        plt.grid(False)
        plt.axis('off')
        plt.imshow(grid, aspect='auto', cmap='viridis')
    plt.show()

In [ ]:
predict()

In [ ]:
visaul_confusion_matrix()

### Define Grad Cam

In [ ]:
def get_img_array(dir, size):
    img = keras.utils.load_img(dir, target_size=size)
    array = keras.utils.img_to_array(img)
    array = np.expand_dims(array)
    return array
def make_gradcam_heatmap(img_array, model, last_conv_layer_name, pred_index=None):
    grad_model = keras.models.Model(
        model.inputs, [model.get_layer(last_conv_layer_name).output, model.output]
    )
    with tf.GradientTape() as tape:
        last_conv_layer_output, preds = grad_model(img_array)
        if pred_index is None:
            pred_index = tf.argmax(preds[0])
        class_channel = preds[:, pred_index]

    grads = tape.gradient(class_channel, last_conv_layer_output)
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))

    last_conv_layer_output = last_conv_layer_output[0]
    heatmap = last_conv_layer_output @ pooled_grads[..., tf.newaxis]
    heatmap = tf.squeeze(heatmap)

    heatmap = tf.maximum(heatmap, 0) / tf.math.reduce_max(heatmap)
    return heatmap.numpy()


In [ ]:
def save_and_display_gradcam(img_path, heatmap, cam_path="cam.jpg", alpha=0.4):
    # Load the original image
    img = keras.utils.load_img(img_path)
    img = keras.utils.img_to_array(img)

    # Rescale heatmap to a range 0-255
    heatmap = np.uint8(255 * heatmap)

    # Use jet colormap to colorize heatmap
    jet = cm.get_cmap("jet")

    # Use RGB values of the colormap
    jet_colors = jet(np.arange(256))[:, :3]
    jet_heatmap = jet_colors[heatmap]

    # Create an image with RGB colorized heatmap
    jet_heatmap = keras.utils.array_to_img(jet_heatmap)
    jet_heatmap = jet_heatmap.resize((img.shape[1], img.shape[0]))
    jet_heatmap = keras.utils.img_to_array(jet_heatmap)

    # Superimpose the heatmap on original image
    superimposed_img = jet_heatmap * alpha + img
    superimposed_img = keras.utils.array_to_img(superimposed_img)

    # Save the superimposed image
    superimposed_img.save(cam_path)

    # Display Grad CAM
    display(Image(cam_path))